In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Differential Log Fold Change (LFC) for mESC_Ag data.

In [ ]:
%load_ext tensorboard

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import tensorflow as tf
import tensorflow_probability as tfp
import time
import pickle
import os

#Load Data
Root_Folder = "/content/drive/MyDrive"
data = "gastr"
data_parent_folder = "/Data"
code_parent_folder = "/scMVI"

os.chdir('/content/drive/My Drive/' + code_parent_folder +'/Code')

Rna_train=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_train_" + data + ".csv",",")
Rna_nrm_train=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_nrm_train_" + data + ".csv",",")

Rna_test=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_test_" + data + ".csv",",")
Rna_nrm_test=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_nrm_test_" + data + ".csv",",")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
MRna_train=Rna_train.drop(columns=Rna_train.columns[0])
MRna_nrm_train=Rna_nrm_train.drop(columns=Rna_nrm_train.columns[0])

MRna_test=Rna_test.drop(columns=Rna_test.columns[0])
MRna_nrm_test=Rna_nrm_test.drop(columns=Rna_nrm_test.columns[0])

In [ ]:
Rna = pd.concat([MRna_train,MRna_test],axis=0)

In [ ]:
Rna_tr=tf.convert_to_tensor(MRna_train,dtype=tf.float32)
Rna_tst=tf.convert_to_tensor(MRna_test,dtype=tf.float32)
Rna_nrm_tr = tf.repeat(tf.math.log(tf.convert_to_tensor(MRna_nrm_train,dtype=tf.float32)),repeats=Rna_tr.shape[1],axis=1)
Rna_nrm_tst= tf.repeat(tf.math.log(tf.convert_to_tensor(MRna_nrm_test,dtype=tf.float32)),repeats=Rna_tr.shape[1],axis=1)

In [ ]:
#Functions for estimating differential directions and stats for experiments outlined in the main chapter 
def index_creation(meta_data,group_by,group_1,smps_group_1,group_2,smps_group_2):


                        cell_num = meta_data.shape[0]
                        group_index_1 = np.zeros((cell_num,))
                        group_index_2 = np.zeros((cell_num,))

                        ind_1 = np.where(meta_data[group_by]==group_1)[0]
                        ind_2 = np.where(meta_data[group_by]==group_2)[0]

                        if np.logical_or(len(ind_1)<smps_group_1,len(ind_2)<smps_group_2):

                                    raise ValueError('samples for groups 1 and 2 cannot be more than cells in groups 1 and 2')        



                        perm_ind_1 = ind_1[np.random.permutation(len(ind_1))[:smps_group_1]]
                        perm_ind_2 = ind_2[np.random.permutation(len(ind_2))[:smps_group_2]]

                        group_index_1[perm_ind_1]=1
                        group_index_2[perm_ind_2]=1

                        group_index_1 = tf.squeeze(tf.where(group_index_1==1))
                        group_index_2 = tf.squeeze(tf.where(group_index_2==1))

                        return group_index_1, group_index_2

def create_exp(metadata,grp_1,smps_1,grp_2,smps_2):

        smps_group_1 =[smps_1,int(np.round(smps_1*0.2)),smps_1,int(np.round(smps_1*0.2)),int(np.round(smps_1*0.5)),int(np.round(smps_1*0.5)),int(np.round(smps_1*0.2))]
        smps_group_2 =[smps_2,smps_2,int(np.round(smps_2*0.2)),int(np.round(smps_2*0.5)),int(np.round(smps_2*0.2)),int(np.round(smps_2*0.5)),int(np.round(smps_2*0.2))]

        col_all_rna = []
        eps=0.0001
        for jj in range(5):


              group_index_11,group_index_21 = index_creation(metadata,'stage',grp_1,smps_group_1[0],grp_2,smps_group_2[0])
              group_index_12,group_index_22 = index_creation(metadata,'stage',grp_1,smps_group_1[1],grp_2,smps_group_2[1])
              group_index_13,group_index_23 = index_creation(metadata,'stage',grp_1,smps_group_1[2],grp_2,smps_group_2[2])
              group_index_14,group_index_24 = index_creation(metadata,'stage',grp_1,smps_group_1[3],grp_2,smps_group_2[3])
              group_index_15,group_index_25 = index_creation(metadata,'stage',grp_1,smps_group_1[4],grp_2,smps_group_2[4])
              group_index_16,group_index_26 = index_creation(metadata,'stage',grp_1,smps_group_1[5],grp_2,smps_group_2[5])
              group_index_17,group_index_27 = index_creation(metadata,'stage',grp_1,smps_group_1[6],grp_2,smps_group_2[6])

              a1_rna = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_11),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_21),axis=0))
              a2_rna = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_12),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_22),axis=0))
              a3_rna = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_13),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_23),axis=0))
              a4_rna = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_14),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_24),axis=0))
              a5_rna = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_15),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_25),axis=0))
              a6_rna = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_16),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_26),axis=0))
              a7_rna = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_17),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_27),axis=0))              


              col_all_rna.append([a1_rna,a2_rna,a3_rna,a4_rna,a5_rna,a6_rna,a7_rna])

              print(100*(1+jj)/5.0)


        return col_all_rna

def collect_stats(x):

        x_med = tf.squeeze(x)
        x_med_mn = tf.reduce_mean(tf.reduce_mean(x_med,axis=1),axis=0)[tf.newaxis,:]
        x_med_std = tf.math.sqrt(tf.reduce_mean(tfp.stats.variance(x_med,sample_axis=1),axis=0))[tf.newaxis,:]

        x_med_rng = tf.reduce_mean(tf.reduce_max(x_med,axis=1) -tf.reduce_min(x_med,axis=1),axis=0)[tf.newaxis,:]

        return x_med_mn,x_med_std,x_med_rng

In [ ]:
Rna_comb = tf.concat([Rna_tr,Rna_tst],axis=0)
Rna_nrm_comb = tf.concat([Rna_nrm_tr,Rna_nrm_tst],axis=0)

xxxx = tf.divide(Rna_comb,tf.math.exp(Rna_nrm_comb))

In [ ]:
#Load Metadata
Rna = pd.concat([Rna_train,Rna_test],axis=0)

gastr_ind_drp = [0,1,2,3,4,5,6,7,8,9]#8 
gastr_QC = pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/sample_metadata_gastr.csv",'\t',index_col=[0])


gastr_QC = gastr_QC.drop(columns=gastr_QC.columns[gastr_ind_drp],axis=1)
gastr_QC.index = gastr_QC.index.str.replace('-','.')

gastr_QC = gastr_QC.loc[Rna.iloc[:,0]]

gastr_ind_drp = [0,1,2,3,4,5,6,7,9,10]#8 
gastr_QC1 = pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/sample_metadata_gastr.csv",'\t',index_col=[0])


gastr_QC1 = gastr_QC1.drop(columns=gastr_QC1.columns[gastr_ind_drp],axis=1)
gastr_QC1.index = gastr_QC1.index.str.replace('-','.')

gastr_QC1 = gastr_QC1.loc[Rna.iloc[:,0]]

In [ ]:
print(gastr_QC1.value_counts())

stage
E7.5     343
E6.5     271
E5.5     108
E4.5     104
dtype: int64


In [ ]:
#Estimates differential directions
lab1 = "E7.5" #(343)
smps_1 = 343
lab2 = "E6.5" #(271)
smps_2 = 271
lab3 = "E5.5" #(108)
smps_3 = 108
lab4 = "E4.5" #(104)
smps_4 = 104

col_all_1_rna = create_exp(gastr_QC1,lab1,smps_1,lab2,smps_2) #col_all_1_met,col_all_1_acc,
col_all_2_rna = create_exp(gastr_QC1,lab2,smps_2,lab3,smps_3) #col_all_2_met,col_all_2_acc,
col_all_3_rna = create_exp(gastr_QC1,lab3,smps_3,lab4,smps_4) #col_all_3_met,col_all_3_acc,

20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0


In [ ]:
cl_1_med_rna,_,cl_1_med_rng_rna = collect_stats(col_all_1_rna)
cl_2_med_rna,_,cl_2_med_rng_rna = collect_stats(col_all_2_rna)
cl_3_med_rna,_,cl_3_med_rng_rna = collect_stats(col_all_3_rna)

In [ ]:
med_1_rna_rng = pd.DataFrame(np.squeeze(cl_1_med_rng_rna),index=Rna.columns[1:])
med_2_rna_rng = pd.DataFrame(np.squeeze(cl_2_med_rng_rna),index=Rna.columns[1:])
med_3_rna_rng= pd.DataFrame(np.squeeze(cl_2_med_rng_rna),index=Rna.columns[1:])

med_1_rna = pd.DataFrame(np.squeeze(cl_1_med_rna),index=Rna.columns[1:])
med_2_rna = pd.DataFrame(np.squeeze(cl_2_med_rna),index=Rna.columns[1:])
med_3_rna = pd.DataFrame(np.squeeze(cl_3_med_rna),index=Rna.columns[1:])

In [ ]:
if os.path.isdir(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/mESC_NMT") == False:
      os.mkdir(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/mESC_NMT")

med_1_rna_rng.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/mESC_NMT/med_1_rng_Differentia_Testing_rna.csv",header=True,sep='\t')
med_2_rna_rng.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/mESC_NMT/med_2_rng_Differentia_Testing_rna.csv",header=True,sep='\t')
med_3_rna_rng.to_csv(Root_Folder + code_parent_folder +"/scMVI_DDLFC_meta/mESC_NMT/med_3_rng_Differentia_Testing_rna.csv",header=True,sep='\t')

med_1_rna.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/mESC_NMT/Diff_Dir_1_Differentia_Testing_rna.csv",header=True,sep='\t')
med_2_rna.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/mESC_NMT/Diff_Dir_2_Differentia_Testing_rna.csv",header=True,sep='\t')
med_3_rna.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/mESC_NMT/Diff_Dir_3_Differentia_Testing_rna.csv",header=True,sep='\t')